# Import libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from IPython.core.debugger import set_trace
import json
from matplotlib import colors
from ast import literal_eval

# Load data

In [2]:
# psynet 11
fname = "natising1"

PATH = f"data/{fname}/"
nodes = pd.read_csv(PATH + "CustomNode.csv", low_memory=False)
# networks = pd.read_csv(PATH + "CustomNetwork.csv", low_memory=False) # old psynet
trials = pd.read_csv(PATH + "CustomTrial.csv", low_memory=False)

In [5]:
print(nodes)

     vars    id        creation_time  failed  failed_reason  time_of_death  \
0      {}    17  2024-07-02 11:50:04   False            NaN            NaN   
1      {}    18  2024-07-02 11:50:04   False            NaN            NaN   
2      {}    19  2024-07-02 11:50:04   False            NaN            NaN   
3      {}    20  2024-07-02 11:50:04   False            NaN            NaN   
4      {}    21  2024-07-02 11:50:04   False            NaN            NaN   
...   ...   ...                  ...     ...            ...            ...   
1095   {}  1368  2024-07-02 16:12:10   False            NaN            NaN   
1096   {}  1369  2024-07-02 16:12:24   False            NaN            NaN   
1097   {}  1370  2024-07-02 16:12:44   False            NaN            NaN   
1098   {}  1371  2024-07-02 16:12:48   False            NaN            NaN   
1099   {}  1372  2024-07-02 16:12:54   False            NaN            NaN   

             type  network_id  participant_id   trial_maker_id 

In [15]:
# filter nodes
node_data = nodes
node_data = node_data[node_data["failed"] == False]
node_data = node_data[["id", "network_id", "degree", "trial_maker_id", "definition", "seed"]]
node_data = node_data.sort_values(["network_id", "degree"])
node_data["definition"].iloc[1]

def find_end_tt(d, end_tt_list=None):
    if end_tt_list is None:
        end_tt_list = []
    if isinstance(d, dict):
        for key, value in d.items():
            if key == "start_tt":
                end_tt_list.append(value)
            elif isinstance(value, (dict, list)):
                find_end_tt(value, end_tt_list)
    elif isinstance(d, list):
        for item in d:
            find_end_tt(item, end_tt_list)
    return end_tt_list


node_data['start_tt'] = node_data['definition'].apply(lambda x: find_end_tt(json.loads(x)))

print(node_data['start_tt'])


0                                                     []
150    [1253.6281179138323, 2036.3265306122448, 3414....
325    [977.6190476190476, 1702.4036281179137, 2550.3...
484    [1231.156462585034, 1899.8866213151928, 2722.4...
866    [987.0975056689342, 1658.2766439909296, 2384.1...
                             ...                        
472    [1181.111111111111, 1815.6462585034014, 2518.4...
522    [1227.6190476190477, 1793.1972789115646, 2382....
574    [1001.7233560090702, 1556.2131519274376, 2160....
594    [932.8571428571429, 1498.0498866213152, 2098.7...
726    [1115.079365079365, 1669.591836734694, 2247.98...
Name: start_tt, Length: 1100, dtype: object


# Preprocess data

In [8]:
# filter nodes
node_data = nodes
node_data = node_data[node_data["failed"] == False]
node_data = node_data[["id", "network_id", "degree", "trial_maker_id", "definition", "seed"]]
node_data = node_data.sort_values(["network_id", "degree"])

# process node definition
mel_id = []
# register = []
trial_type = []
target_pitches = []
# reference_pitch = []
target_intervals = []
target_note_durations = []
# target_ISIs = []

for i in range(len(node_data)):
    print()
    # definition = node_data["definition"].to_numpy()[i]
    # definition = literal_eval(definition)
    # definition = [d["content"] for d in definition if d["is_center"]][0]

    definition_str = node_data["definition"].iloc[i]  # Access the string directly
    print(definition_str)
    definition = json.loads(definition_str)  # Parse JSON string into dictionary
    # definition = [d["content"] for d in definition if d["is_center"]][0]
    
    # mel_id.append(definition["mel_id"])
    # register.append(definition["register"])
    trial_type.append(definition["trial_type"])
    target_pitches.append(definition["target_pitches"])
    # reference_pitch.append(definition["reference_pitch"])
    target_intervals.append(definition["target_intervals"])
    target_note_durations.append(definition["target_note_durations"])
    # target_ISIs.append(definition["target_ISIs"])

# node_data["mel_id"] = mel_id
# node_data["register"] = register
node_data["trial_type"] = trial_type
node_data["target_pitches"] = target_pitches
# node_data["reference_pitch"] = reference_pitch
node_data["target_intervals"] = target_intervals
node_data["target_note_durations"] = target_note_durations
# node_data["target_ISIs"] = target_ISIs

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [51]:
node_data

id  network_id  degree   trial_maker_id  \
0      17          17       0  imitation_chain   
150   184          17       1  imitation_chain   
325   401          17       2  imitation_chain   
484   607          17       3  imitation_chain   
866  1075          17       4  imitation_chain   
..    ...         ...     ...              ...   
472   575         116       6  imitation_chain   
522   649         116       7  imitation_chain   
574   711         116       8  imitation_chain   
594   733         116       9  imitation_chain   
726   900         116      10  imitation_chain   

                                            definition  \
0    {"register": "high", "reference_pitch": 61.284...   
150  {"num_target_pitches": 5, "target_pitches": [4...   
325  {"num_target_pitches": 5, "target_pitches": [4...   
484  {"num_target_pitches": 5, "target_pitches": [4...   
866  {"num_target_pitches": 5, "target_pitches": [4...   
..                                                 ...   
472  {"num_target_pitches": 5, "target_pitches": [5...   
522  {"num_target_pitches": 5, "target_pitches": [5...   
574  {"num_target_pitches": 5, "target_pitches": [4...   
594  {"num_target_pitches": 5, "target_pitches": [4...   
726  {"num_target_pitches": 5, "target_pitches": [5...   

                                                  seed    trial_type  \
0    {"register": "high", "reference_pitch": 61.284...  source_trial   
150  {"num_target_pitches": 5, "target_pitches": [4...    node_trial   
325  {"num_target_pitches": 5, "target_pitches": [4...    node_trial   
484  {"num_target_pitches": 5, "target_pitches": [4...    node_trial   
866  {"num_target_pitches": 5, "target_pitches": [4...    node_trial   
..                                                 ...           ...   
472  {"num_target_pitches": 5, "target_pitches": [5...    node_trial   
522  {"num_target_pitches": 5, "target_pitches": [5...    node_trial   
574  {"num_target_pitches": 5, "target_pitches": [4...    node_trial   
594  {"num_target_pitches": 5, "target_pitches": [4...    node_trial   
726  {"num_target_pitches": 5, "target_pitches": [5...    node_trial   

                                        target_pitches  \
0    [57.67352691129944, 62.356609064111915, 54.015...   
150  [41.2301949207222, 47.73754102979737, 46.26591...   
325  [42.17087190612439, 48.2762168046018, 39.91640...   
484  [46.54707338409966, 49.67340939243262, 43.6234...   
866  [49.31820273391566, 46.78752903365064, 52.1746...   
..                                                 ...   
472  [57.726561446079174, 58.8575062588259, 64.7178...   
522  [58.131089353852104, 59.787489440268885, 62.18...   
574  [42.756865465825626, 45.108898580266995, 50.79...   
594  [43.86270838950719, 46.51439516176795, 49.8302...   
726  [54.837401041731994, 57.70516655543797, 61.811...   

                                      target_intervals  \
0    [4.683082152812474, -8.341267397002184, 3.3417...   
150  [6.507346109075172, -1.4716222653180253, -9.40...   
325  [6.105344898477412, -8.359807463784954, 2.1551...   
484  [3.126336008332956, -6.0499734660868825, -0.61...   
866  [-2.5306737002650195, 5.387138732632721, -6.24...   
..                                                 ...   
472  [1.1309448127467263, 5.860332080412867, -10.19...   
522  [1.656400086416781, 2.399038286228823, -2.7713...   
574  [2.352033114441369, 5.683967877947396, -3.7242...   
594  [2.65168677226076, 3.315816768143776, -4.93341...   
726  [2.867765513705976, 4.106826910014853, -4.6476...   

                                 target_note_durations  \
0    [0.917913809592499, 0.6635450743593847, 1.0978...   
150  [0.5256009070294784, 0.530453514739229, 0.3980...   
325  [0.5724263038548753, 0.6006802721088438, 0.482...   
484  [0.34537414965986407, 0.4519047619047619, 0.38...   
866  [0.3563718820861677, 0.4600453514739229, 0.464...   
..                                                 ...   
472  [0.5465532879818595, 0.6167800453514738

# Export node data to csv

In [52]:
node_data.to_csv(f"{fname}_node_data.csv")

# Trial level data

In [53]:
# filter trials
def extract_valence_arousal(answer):
    try:
        if isinstance(answer, str):
            data = json.loads(answer)
            valence = data.get('valence', None)
            arousal = data.get('arousal', None)
            return valence, arousal
        else:
            return None, None
    except (KeyError, TypeError, json.JSONDecodeError) as e:
        return None, None

trial_data = trials
trial_data = trial_data[trial_data["failed"] == False]

# Apply the function to both possible columns
valence_arousal_seed = trial_data['listen_and_rate_trial_seed'].apply(extract_valence_arousal)
valence_arousal_trial = trial_data['listen_and_rate_trial'].apply(extract_valence_arousal)

# Combine the results, prioritizing 'seed' over 'trial'
trial_data['valence'] = valence_arousal_seed.apply(lambda x: x[0] if x[0] is not None else None).combine_first(valence_arousal_trial.apply(lambda x: x[0]))
trial_data['arousal'] = valence_arousal_seed.apply(lambda x: x[1] if x[1] is not None else None).combine_first(valence_arousal_trial.apply(lambda x: x[1]))

degrees = []

for i in range(len(trial_data)):
    current_node_id = trial_data.iloc[i]["origin_id"]
    current_nodes = node_data[node_data["id"] == current_node_id]
    current_degree = current_nodes["degree"].to_numpy()[0]
    # answer = d['listen_and_rate_trial_seed']['valence']
    # current_vertex_id = current_nodes["vertex_id"].to_numpy()[0]
    # current_dependent_vertex_ids = current_nodes["dependent_vertex_ids"].to_numpy()[0]
    degrees.append(current_degree)
    # vertex_ids.append(current_vertex_id)
    # dependent_vertex_ids.append(current_dependent_vertex_ids)

trial_data["degree"] = degrees

trial_data = trial_data[["id", "network_id", "degree", "trial_maker_id", "origin_id", "participant_id", "arousal", "valence", "analysis"]]
trial_data = trial_data.sort_values(["network_id", "degree"])

/var/folders/v7/s8wj18ld78gdbf0vgx1ptf7r0000gr/T/ipykernel_93173/4272298720.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trial_data['valence'] = valence_arousal_seed.apply(lambda x: x[0] if x[0] is not None else None).combine_first(valence_arousal_trial.apply(lambda x: x[0]))
/var/folders/v7/s8wj18ld78gdbf0vgx1ptf7r0000gr/T/ipykernel_93173/4272298720.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trial_data['arousal'] = valence_arousal_seed.apply(lambda x: x[1] if x[1] is not None else None).c

In [54]:
trial_data

id  network_id  degree   trial_maker_id  origin_id  participant_id  \
68    131          17       0  imitation_chain         17               7   
283   500          17       1  imitation_chain        184              24   
469   886          17       2  imitation_chain        401              40   
928  1633          17       3  imitation_chain        607              62   
980  1698          17       4  imitation_chain       1075              72   
..    ...         ...     ...              ...        ...             ...   
455   796         116       5  imitation_chain        521              36   
508   946         116       6  imitation_chain        575              41   
568  1032         116       7  imitation_chain        649              45   
591  1056         116       8  imitation_chain        711              48   
756  1343         116       9  imitation_chain        733              59   

    arousal valence                                           analysis  
68        3       4  {"failed": false, "reason": "All good", "regis...  
283       5       4  {"failed": false, "reason": "All good", "regis...  
469       4       4  {"failed": false, "reason": "All good", "regis...  
928       5       6  {"failed": false, "reason": "All good", "regis...  
980       1       4  {"failed": false, "reason": "All good", "regis...  
..      ...     ...                                                ...  
455       7      10  {"failed": false, "reason": "All good", "regis...  
508       2       6  {"failed": false, "reason": "All good", "regis...  
568       1       6  {"failed": false, "reason": "All good", "regis...  
591       4       5  {"failed": false, "reason": "All good", "regis...  
756       7       2  {"failed": false, "reason": "All good", "regis...  

[1000 rows x 9 columns]

# Export trial data to csv

In [55]:
trial_data.to_csv(f"{fname}_trial_data.csv")

# Experiment Cost and Demographics

In [56]:
participants = pd.read_csv(PATH + "participant.csv", low_memory=False)

In [57]:
valid_participants = participants[participants["complete"] == True]
valid_participants = valid_participants[valid_participants["failed"] == False]
experiment_summary = {
    "N_participants": valid_participants.shape[0],
    "cost": participants["base_pay"].sum() + participants["bonus"].sum()
}

experiment_summary

{'N_participants': 67, 'cost': np.float64(226.20000000000002)}